In [1]:
is_installed <- function(mypkg){ is.element(mypkg, installed.packages()[,1])}
load_or_install<-function(package_names){
    r = getOption("repos")
    r["CRAN"] = "http://cran.us.r-project.org"
    options(repos = r)
    #https://www.r-bloggers.com/2012/05/loading-andor-installing-packages-programmatically/
    #quick install or load packages
  for(package_name in package_names)
  {
    if(!is_installed(package_name))
    {
       message(sprintf("installing package: %s",package_name))
       install.packages(package_name,repos = "http://cran.us.r-project.org")
    }
    else{
        message(sprintf("loading package: %s",package_name))
        library(package_name,character.only=TRUE,quietly=TRUE,verbose=FALSE)
    }
  }
}

In [2]:

load_or_install(c("DT","grid","plyr","dplyr","ggplot2","plotly","knitr","future","patchwork","gridExtra","RColorBrewer","data.table","extrafont"))

loading package: DT

loading package: grid

loading package: plyr

loading package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


loading package: ggplot2

loading package: plotly


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following objects are masked from ‘package:plyr’:

    arrange, mutate, rename, summarise


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout


loading package: knitr

loading package: future

loading package: patchwork

loading package: gridExtra


Attaching package: ‘gridExtra’


The following object is masked from ‘package

In [ ]:
# snakemake params

color_names = snakemake@params['color_names']
color_hex = snakemake@params['color_hex']
clusters_of_interest = snakemake@params['clusters_of_interest']
out_file = snakemake@output['output_file']
input_file = snakemake@input['input_tsv']
color_tsv = snakemake@input['color_tsv']



# Load data

In [ ]:
data_f  = read.delim(file.path(getwd(),input_file), sep = "\t")#, fileEncoding = 'utf-8')

plot.meta = data.table(data_f)

In [ ]:
#get updated color palette
# this is due to colors being similar
color_tsv = file.path(getwd(),color_tsv)
color_tsv_file = read.delim(color_tsv)

In [5]:
#color palettes
cluster_colors = unlist(color_tsv_file[1,-1])
names(cluster_colors) = gsub("\\.","-",names(cluster_colors))
names(cluster_colors) = gsub("-Eryth"," Eryth",names(cluster_colors))


In [ ]:
#get valid colors
color_names <- unlist(color_names)
color_hex <- unlist(color_hex)

names(color_hex) <- color_names

In [ ]:
clusters_of_interest <- unlist(clusters_of_interest)

In [20]:
data_f  = read.delim("/home/groups/CEDAR/enright/Runx_AML/submission_pipeline/scRNAvelocity/results/v3.unenriched/scvelo_obs.tsv", sep = "\t")#, fileEncoding = 'utf-8')

plot.meta = data.table(data_f)

In [21]:
#override clusters of interest
clusters_of_interest <- c("HCA","Progenitor","GMP","MKP","Mono")

# Process Table

In [22]:
process_table <- function(plot.meta,pseudotime = 'dpt_pseudotime',clusters_of_interest = NULL,b_scale = 1,binwidth = 0.02, alpha_var = 1){
    #takes a data table
    #returns a table to be saved
    
    set(plot.meta,j = 'pseudotime',value = plot.meta[[pseudotime]])
    
    #use all clusters if clusters of interest is null
    if (is.null(clusters_of_interest)){
        clusters_of_interest <- unique(plot.meta$clusters)
    }
    
    #subset
    myplotmeta = plot.meta[clusters %in% clusters_of_interest,]
    #get bins for the pseudotime
    myplotmeta[, ps.bin := ((findInterval(pseudotime*b_scale, seq(0, 1, by = binwidth))-1)*binwidth)]
    #calculate total number of cells per Condition
    summarycount <- plot.meta[, .N, by = .(Condition)]
    #rename for next step
    colnames(summarycount)[colnames(summarycount) == "N"] = "N.Condition"
    #get percentage of number of cells across pseudotime
    meta_bin <- myplotmeta[, .N, by = .(ps.bin, Condition)]
    meta_bin <- merge(x = meta_bin, y = summarycount, by = "Condition") #adds number of cells per condition column to metabin
    meta_bin <- meta_bin[, pct_condition := N/N.Condition*100][order(ps.bin)]
    return(meta_bin)
}    

# Line plot: percent of total cells across pseudotime 

In [12]:
# change font type
dev.new(family = "Arial")

In [1]:
#function to perform plotting
#plot.meta is a data table with at least three columns for each cell: 'pseudotime',Condition,clusters
plot_pseudotime <- function(plot.meta,pseudotime = 'dpt_pseudotime',x_lab = "Diffusion Pseudotime",y_lab="% of Total Cells",title_name = "" ,clusters_of_interest = NULL,color_hex = NULL,cluster_colors = NULL,b_scale = 1,binwidth = 0.02, alpha_var = 1,save_file = NULL){
    #takes a data table
    #returns a plot or saves a plot if 'save_file' is a path to a file and not null
    
    #alpha_var for color opacity
    #add pseudotime to table based on the user defined pseudotime of interest
    #default is 'dpt_pseudotime'
    set(plot.meta,j = 'pseudotime',value = plot.meta[[pseudotime]])
    
    #use all clusters if clusters of interest is null
    if (is.null(clusters_of_interest)){
        clusters_of_interest <- unique(plot.meta$clusters)
    }
    #correct colors for condition if there is no input
    if (is.null(color_hex)){
        
        color_names = unique(plot.meta$Condition)
        color_hex <- brewer.pal(max(3,length(color_names)),"Dark2")
        color_hex <- color_hex[seq(length(color_names))]
        names(color_hex) <- color_names
    }
    
    #subset
    myplotmeta = plot.meta[clusters %in% clusters_of_interest,]
    #correct colors for clusters if no cluster colors given
    if (is.null(cluster_colors)){
        
        cluster_names = unique(myplotmeta$clusters)
        cluster_colors <- brewer.pal(length(unique(cluster_names)),"Set3")
        names(cluster_colors) <- cluster_names
    }
    #get bins for the pseudotime
    myplotmeta[, ps.bin := ((findInterval(pseudotime*b_scale, seq(0, 1, by = binwidth))-1)*binwidth)]
    #calculate total number of cells per Condition
    summarycount <- plot.meta[, .N, by = .(Condition)]
    #rename for next step
    colnames(summarycount)[colnames(summarycount) == "N"] = "N.Condition"
    #get percentage of number of cells across pseudotime
    meta_bin <- myplotmeta[, .N, by = .(ps.bin, Condition)]
    meta_bin <- merge(x = meta_bin, y = summarycount, by = "Condition")
    meta_bin <- meta_bin[, pct_condition := N/N.Condition*100][order(ps.bin)]
    
    xlim <- c(0, ceiling(max(myplotmeta$pseudotime)))
    ylim <- c(0, ceiling(max(meta_bin$pct_condition))+2)
    mar_offset = c(-1.5, 2.,-3,4)
    par(mar = c(5.1, 4.1, 6.1, 2.1) + mar_offset)
    if (!is.null(save_file)){
        pdf(save_file)
    }
    pplot <- plot(NULL, xlab = "", ylab = "", xlim = xlim, ylim = ylim, las = 1, yaxt = "n",main = title_name)

    mtext(bquote(bold(.(y_lab))), side = 2, line = 3, at = floor(max(ylim)/2), cex = 1.2)
    mtext(bquote(bold(.(x_lab))), side = 1, line = 2.5, cex = 1.2)
    abline(h=0)
    #abline(h=seq(0.5, max(ylim), 0.5), col = "lightgrey") #grey lines every 0.5 along the yaxis
    axis(2, at = seq(0,floor(max(ylim)),by = 1), labels = seq(0,floor(max(ylim)),by = 1), tick = TRUE, las = TRUE)

    # Plot % cells
    for (myline in color_names) {
          meta_bin[Condition == myline, points(ps.bin, pct_condition, col = alpha(color_hex[[myline]],alpha_var), pch = 20, cex = 1.5)]
          meta_bin[Condition == myline, lines(spline(ps.bin, pct_condition, method = "n"), col = alpha(color_hex[[myline]],alpha_var), lwd = 2.5)]
        }


    #Draw Rug of cell type
    # done in a way to have the least amount of cells per cluster on top of bigger clusters
    for (clustr in rev(names(sort(table(myplotmeta$cluster))))){
        myrugmeta <- myplotmeta[cluster == clustr]
        myrugmeta[, points(x=pseudotime, y = rep(max(ylim),nrow(myrugmeta)),pch = "|",cex =1,col = cluster_colors[[clustr]])]
    }
    
    # Legend for lines & rug
    pctLegend <- color_names
    legendcolor <- color_hex[color_names]
    leg <- legend("topright",inset = c(0.2,0.05),horiz = FALSE, legend = pctLegend, col = legendcolor, lwd = 2.5, cex = 1.2, bty = "n", plot = T)
    pctLegend <- clusters_of_interest
    legendcolor <- unname(cluster_colors[clusters_of_interest])
    legend(x = leg$rect$left, y = leg$rect$top - leg$rect$h,horiz = FALSE, legend = pctLegend, col = legendcolor, pch = 15, cex = 1.2, bty = "n", plot = T)
    if (!is.null(save_file)){
        dev.off()
    }
    else{
        return(pplot)
    }
}

### Save plots for various of pseudotime approaches

In [ ]:
pseudotime <- 'dpt_pseudotime'
plot_dpt <- file.path(getwd(),out_file)
plot_pseudotime(plot.meta,pseudotime = pseudotime, x_lab = "Diffusion Pseudotime",title_name = '',clusters_of_interest = clusters_of_interest,color_hex = color_hex,cluster_colors = cluster_colors,b_scale = 1,binwidth = 0.02, save_file = plot_dpt)

In [ ]:
#save table
df_table <- process_table(plot.meta,pseudotime = pseudotime,clusters_of_interest = clusters_of_interest,b_scale = 1,binwidth = 0.02)
out_table <- file.path(dirname(plot_dpt),sprintf("pseudotime_%s_table.tsv",pseudotime))
write.table(df_table,file = out_table, quote = FALSE, sep = "\t")

In [ ]:
pseudotime <- 'latent_time'
plot_dpt <- dirname(file.path(getwd(),out_file))
plot_dpt <- file.path(plot_dpt,sprintf("pseudotime_%s_r.pdf",pseudotime))
plot_pseudotime(plot.meta,pseudotime = pseudotime, x_lab = "Latent Time",title_name = '',clusters_of_interest = clusters_of_interest,color_hex = color_hex,cluster_colors = cluster_colors,b_scale = 1,binwidth = 0.02, save_file = plot_dpt)

In [ ]:
#save table
df_table <- process_table(plot.meta,pseudotime = pseudotime,clusters_of_interest = clusters_of_interest,b_scale = 1,binwidth = 0.02)
out_table <- file.path(dirname(plot_dpt),sprintf("pseudotime_%s_table.tsv",pseudotime))
write.table(df_table,file = out_table, quote = FALSE, sep = "\t")

In [ ]:
pseudotime <- 'velocity_pseudotime'
plot_dpt <- dirname(file.path(getwd(),out_file))
plot_dpt <- file.path(plot_dpt,sprintf("pseudotime_%s_r.pdf",pseudotime))
plot_pseudotime(plot.meta,pseudotime = pseudotime, x_lab = "Velocity Pseudotime",title_name = '',clusters_of_interest = clusters_of_interest,color_hex = color_hex,cluster_colors = cluster_colors,b_scale = 1,binwidth = 0.02, save_file = plot_dpt)

In [ ]:
#save table
df_table <- process_table(plot.meta,pseudotime = pseudotime,clusters_of_interest = clusters_of_interest,b_scale = 1,binwidth = 0.02)
out_table <- file.path(dirname(plot_dpt),sprintf("pseudotime_%s_table.tsv",pseudotime))
write.table(df_table,file = out_table, quote = FALSE, sep = "\t")

#### session info

In [109]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /home/groups/precepts/enright/miniconda3/envs/seurat_note/lib/libmkl_rt.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] data.table_1.13.6  gridExtra_2.3      patchwork_1.1.1    future_1.21.0     
 [5] knitr_1.31         plotly_4.9.3       ggplot2_3.3.3      dplyr_1.0.6       
 [9] plyr_1.8.6         DT_0.17            SeuratObject_4.0.0 Seurat_4.0.0      

loaded via a namespace (and not 